<a href="https://colab.research.google.com/github/jhonatanyara/SenalesySistemas/blob/main/Ejercicios_parcial_1_SYS_JhonatanYaraLopez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fft, fftfreq

EJERCICIO #1

In [ ]:
# === Punto 1: Distancia media entre x1(t)=A e^{-jn w0 t} y x2(t)=B e^{+jm w0 t} ==
# ------------------ Parámetros (cámbialos aquí) ------------------
A = 2.0
B = 1.5
n = 3
m = -3            # pon m=2 para el caso n+m ≠ 0
w0 = 2*np.pi      # rad/s  -> T0 = 1 s

# ------------------ Simulación para el promedio temporal ------------------
T0 = 2*np.pi/w0
periodos_prom = 2000                 # muchos periodos para promediar bien
fs = 5000                            # Hz (muestras por segundo)
t = np.arange(0.0, periodos_prom*T0, 1/fs)

x1 = A*np.exp(-1j*n*w0*t)            # A e^{-jn w0 t}
x2 = B*np.exp(+1j*m*w0*t)            # B e^{+jm w0 t}

# Distancia media numérica
d2_num = np.mean(np.abs(x1 - x2)**2)
d_num  = np.sqrt(d2_num)

# Distancia teórica
delta = 1.0 if (n + m) == 0 else 0.0
d2_teo = A**2 + B**2 - 2*A*B*delta
d_teo  = np.sqrt(d2_teo)

print("=== Distancia media ===")
print(f"Parámetros: A={A}, B={B}, n={n}, m={m}, w0={w0:.3f} rad/s (T0={T0:.3f} s)")
print(f"d_num  ≈ {d_num:.6f}   (promedio temporal)")
print(f"d_teo  = {d_teo:.6f}   (A^2 + B^2 - 2AB*δ[n+m])")

# ------------------ Visualización (un tramo corto) ------------------
# Tomamos 3 periodos para mostrar
periodos_plot = 3
Nplot = int(periodos_plot*T0*fs)
tp = t[:Nplot]
x1p, x2p = x1[:Nplot], x2[:Nplot]

plt.figure(figsize=(10,4))
plt.plot(tp, np.real(x1p), label='Re{x1(t)}')
plt.plot(tp, np.real(x2p), label='Re{x2(t)}', linestyle='--')
plt.xlabel('t [s]'); plt.ylabel('amplitud')
plt.title('Partes reales de x1(t) y x2(t) (tramo corto)')
plt.grid(True); plt.legend()
plt.tight_layout()
plt.show()

# ------------------ Comprobación del término cruzado (opcional corto) ------------------
# Promedio de e^{-j(n+m)w0 t}; debe ser ~0 si n+m≠0, o 1 si n+m=0
avg_cross = np.mean(np.exp(-1j*(n+m)*w0*t))
print(f"Promedio ⟨e^(-j(n+m)w0 t)⟩ ≈ {avg_cross:.3e}  -> Re ≈ {np.real(avg_cross):.3e}")


EJERCICIO #2

In [ ]:
# =======================
#  x(t) -> muestreo + cuantización (Colab)
#  Señal: x(t)=3 cos(1000π t)+5 sin(3000π t)+10 cos(11000π t)
#  Casos: fs=5 kHz (alias) y fs=12 kHz (adecuado)
# =======================

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
pi = np.pi

# ---------- Señal continua ----------
def x_ct(t):
    return 3*np.cos(1000*pi*t) + 5*np.sin(3000*pi*t) + 10*np.cos(11000*pi*t)

T0   = 1/500          # periodo fundamental
Tsim = 3*T0           # simular >= 3 periodos
t_f  = np.linspace(0, Tsim, 4000)
x_f  = x_ct(t_f)

# ---------- Utilidades ----------
def muestrear(fs):
    t_n = np.arange(0, Tsim, 1/fs)
    x_n = x_ct(t_n)
    return t_n, x_n

def cuantizar_4bits(x, VFS=18.0):
    L = 16
    Delta = 2*VFS/(L-1)       # paso (mid-tread)
    k = np.round(x/Delta)
    k = np.clip(k, -(L-1)/2, (L-1)/2)
    x_q = Delta*k
    Nq   = np.mean((x - x_q)**2) + 1e-18
    Psig = np.mean(x_q**2) + 1e-18
    SQNR = 10*np.log10(Psig/Nq)
    return x_q, Delta, SQNR

def stem_label(x, y, color='C0', marker='o', label=None):
    """Versión compatible de stem (sin use_line_collection) con etiqueta."""
    m, s, b = plt.stem(x, y, linefmt=color+'-', markerfmt=color+marker, basefmt=' ')
    if label:
        m.set_label(label)
    return m, s, b

# ---------- Caso enunciado: fs = 5 kHz (con alias) ----------
fs1 = 5000.0
t1, x1 = muestrear(fs1)

w1 = 2*pi*500/fs1
w2 = 2*pi*1500/fs1
w3 = 2*pi*5500/fs1
w3_mod = w3 % (2*pi)
w3_alias = 2*pi - w3_mod if w3_mod > pi else w3_mod

expr_5k = "x[n] = 13·cos(0.2π n) + 5·sin(0.6π n)"
x1q, Delta1, SQNR1 = cuantizar_4bits(x1, VFS=18.0)

print("=== fs = 5 kHz (con alias) ===")
print(f"w1 = {w1/pi:.3f}·π,  w2 = {w2/pi:.3f}·π,  w3 = {w3/pi:.3f}·π  -> alias = {w3_alias/pi:.3f}·π")
print(expr_5k)
print(f"Cuantización 4 bits: Δ = {Delta1:.2f},  SQNR ≈ {SQNR1:.2f} dB")

plt.figure(figsize=(10,3))
plt.plot(t_f*1e3, x_f, label='x(t) continuo')
stem_label(t1*1e3, x1,  'C1', 'o', label='x[n] 5 kHz')
stem_label(t1*1e3, x1q, 'C2', 's', label='cuantizado 4 bits')
plt.xlabel('t [ms]'); plt.ylabel('amplitud'); plt.grid(True); plt.legend()
plt.title('fs = 5 kHz (alias 5.5 kHz → 0.5 kHz) + cuantización 4 bits')
plt.show()

# ---------- Conversor adecuado: fs = 12 kHz (sin alias) ----------
fs2 = 12000.0
t2, x2 = muestrear(fs2)

w1_2 = 2*pi*500/fs2      # π/12
w2_2 = 2*pi*1500/fs2     # π/4
w3_2 = 2*pi*5500/fs2     # 11π/12
expr_12k = "x[n] = 3·cos(π/12·n) + 5·sin(π/4·n) + 10·cos(11π/12·n)"

print("\n=== fs = 12 kHz (adecuado, sin alias) ===")
print(f"w1 = {w1_2/pi:.3f}·π,  w2 = {w2_2/pi:.3f}·π,  w3 = {w3_2/pi:.3f}·π")
print(expr_12k)

plt.figure(figsize=(10,3))
plt.plot(t_f*1e3, x_f, label='x(t) continuo')
stem_label(t2*1e3, x2, 'C3', 'o', label='x[n] 12 kHz')
plt.xlabel('t [ms]'); plt.ylabel('amplitud'); plt.grid(True); plt.legend()
plt.title('fs = 12 kHz (sin alias) — conversor adecuado')
plt.show()

Ejercicio #4

In [ ]:
# --- Señal "M" con márgenes en 0 fuera del trapecio ---

A  = 1.0   # altura de los picos
d1 = 1.0   # posición del pico (0 < d1 < d2)
d2 = 2.0   # fin del trapecio (0 en ±d2)

# PERÍODO INDEPENDIENTE (debe cumplir T/2 > d2)
T  = 6.0                 # <-- elige T; por ejemplo 6 s (así T/2=3 > d2=2)
assert (T/2) > d2, "Debes escoger T/2 > d2 para tener márgenes en 0."

Fo = 1/T
Fs = 1000*Fo
tv = np.arange(-T/2, T/2, 1/Fs)
Nm = len(tv)

x = np.zeros_like(tv)

# [-d2, -d1]: sube 0 -> A
m1 = (tv >= -d2) & (tv <= -d1)
x[m1] = A*(tv[m1] + d2)/(d2 - d1)

# [-d1, 0]: baja A -> 0
m2 = (tv > -d1) & (tv <= 0)
x[m2] = A*(-tv[m2]/d1)

# [0, d1]: sube 0 -> A
m3 = (tv >= 0) & (tv <= d1)
x[m3] = A*(tv[m3]/d1)

# [d1, d2]: baja A -> 0
m4 = (tv >= d1) & (tv <= d2)
x[m4] = A*(d2 - tv[m4])/(d2 - d1)

# márgenes fuera del trapecio ya quedan en 0:
# [-T/2, -d2] y [d2, T/2] -> x = 0 (por construcción)

# asegurar valores exactos en puntos clave
x[np.isclose(tv, -d2)] = 0.0
x[np.isclose(tv,  d2)] = 0.0
x[np.isclose(tv,   0)] = 0.0
x[np.isclose(tv, -d1)] = A
x[np.isclose(tv,  d1)] = A

# graficar
plt.plot(tv, x, 'r', linewidth=4)
for v in [-T/2, -d2, -d1, 0, d1, d2, T/2]:
    plt.axvline(v, color='k', alpha=0.2)
plt.grid(); plt.xlabel("t[s]", fontsize=14); plt.ylabel("x(t)", fontsize=14)
plt.show()


In [ ]:
N  = 5                 # número de armónicos a cada lado
wo = 2*np.pi/T         # frecuencia fundamental
dt = 1/Fs              # paso temporal para la integral

# definir bases φ_n(t) = e^{j n w0 t} en el intervalo de interés
phin = np.zeros((Nm, 2*N+1), np.complex128)
nv   = np.arange(-N, N+1)          # índices n

for idx, n in enumerate(nv):
    phin[:, idx] = np.exp(1j*n*wo*tv)

# ---- calcular Cn por proyección: Cn = (1/T) ∫ x(t) e^{-jn w0 t} dt  ----
Cn = np.zeros(2*N+1, np.complex128)
for idx, n in enumerate(nv):
    integrando = x * np.exp(-1j*n*wo*tv)
    Cn[idx] = (1.0/T) * np.trapz(integrando, tv)   # trapecios

# (opcional) nivel DC verificación
# Cn[N] ya quedó calculado con n=0; no hace falta fijarlo a mano


In [ ]:
fig = plt.figure(figsize=(8,8))

plt.subplot(2,2,1)
plt.stem(nv, np.real(Cn), 'r', basefmt="k")
plt.xlabel(r'$n$', fontsize=14)
plt.ylabel(r'$Re\{C_n\}$', fontsize=14)
plt.grid(True)

plt.subplot(2,2,2)
plt.stem(nv, np.imag(Cn), 'r', basefmt="k")
plt.xlabel(r'$n$', fontsize=14)
plt.ylabel(r'$Im\{C_n\}$', fontsize=14)
plt.grid(True)

plt.subplot(2,2,3)
plt.stem(nv, np.abs(Cn), 'r', basefmt="k") # magnitud
plt.xlabel(r'$n$', fontsize=14)
plt.ylabel(r'$|C_n|$', fontsize=14)
plt.grid(True)

plt.subplot(2,2,4)
plt.stem(nv, np.angle(Cn), 'r', basefmt="k") # fase
plt.xlabel(r'$n$', fontsize=14)
plt.ylabel(r'$\angle C_n$', fontsize=14)
plt.grid(True)

fig.tight_layout()
plt.show()


In [ ]:
# Potencia media de x(t) para el error relativo: Px = (1/T) ∫ |x(t)|^2 dt
Px = (1.0/T) * np.trapz(np.abs(x)**2, tv)

def pltest(Na=1):
    ind = np.arange(N-Na, N+Na+1)           # armónicos usados
    er = 1 - np.sum(np.abs(Cn[ind])**2) / Px # Parseval discreto (aprox)
    # señal reconstruida
    xe = phin[:, ind].dot(Cn[ind])
    plt.plot(tv, xe, color='b', label='$x_e(t)$')
    plt.plot(tv, x,  color='r', label='x(t)')
    plt.title('$E_r$=%.2f[%%]' % (100*er), fontsize=16)
    plt.xlabel('t[s]'); plt.ylabel('x(t)')
    plt.grid(True); plt.legend(); plt.show()
    return


In [ ]:
from matplotlib import animation, rc
from IPython.display import HTML

fig, ax = plt.subplots(2, 3, figsize=(12,5))

def update(n):
    # limpiar
    for i in range(2):
        for j in range(3):
            ax[i,j].clear()

    # espectro y selección
    ind = np.arange(N-n, N+n+1)
    ax[0,0].stem(nv, np.abs(Cn), 'b', basefmt="k")
    ax[0,0].stem(nv[ind], np.abs(Cn[ind]), 'r', basefmt="k")
    ax[0,0].set_xlabel(r'$n$'); ax[0,0].set_ylabel(r'$|C_n|$')

    # componentes espectrales (real / imag)
    ce = phin[:, ind].dot(np.diag(Cn[ind]))
    ax[0,1].plot(tv, np.real(ce))
    ax[0,1].set_xlabel('t[s]'); ax[0,1].set_ylabel('$Re\{c_n\\phi_n(t)\}$')

    ax[0,2].plot(tv, np.imag(ce))
    ax[0,2].set_xlabel('t[s]'); ax[0,2].set_ylabel('$Im\{c_n\\phi_n(t)\}$')

    # reconstrucción y error temporal
    xe = phin[:, ind].dot(Cn[ind])
    et = x - xe
    er = 1 - np.sum(np.abs(Cn[ind])**2) / Px

    ax[1,1].plot(tv, x,  'g', linewidth=4, label='$x(t)$')
    ax[1,1].plot(tv, xe, 'c', label='$x_e(t)$')
    ax[1,1].plot(tv, et, 'k', label='$e(t)$')
    ax[1,1].set_title('$E_r$=%.2f[%%]' % (100*er))
    ax[1,1].set_xlabel('t[s]'); ax[1,1].set_ylabel('x(t)')
    ax[1,1].legend(loc='center left', bbox_to_anchor=(1, 0.5))

    ax[1,0].axis('off'); ax[1,2].axis('off')
    plt.subplots_adjust(wspace=0.5, hspace=0.7)

anim = animation.FuncAnimation(fig, update, frames=np.arange(0, N+1, 1), blit=False)
rc('animation', html='jshtml')
anim
